<a href="https://colab.research.google.com/github/VanitasBlade/IBM-Gen-AI-Project-Room-Decorator/blob/main/IBM_gen_ai_PROJECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing related Libraries and Dependencies


In [ ]:
!pip install -q diffusers transformers accelerate safetensors
!pip install -q streamlit pyngrok


In [ ]:
from diffusers import StableDiffusionImg2ImgPipeline, StableDiffusionControlNetImg2ImgPipeline, ControlNetModel
import torch
from PIL import Image
import streamlit as st
from pyngrok import ngrok
import io
import numpy as np
import cv2
import os

In [ ]:
%%writefile app.py
# ----------------------------------------
# 🛋️ Smart AI Room Decorator with Streamlit
# ----------------------------------------
# Dynamically chooses between Canny and Depth-based ControlNet pipelines
# based on user-selected design mode (preserve layout vs full redesign)
# ----------------------------------------

# Configure Streamlit page
st.set_page_config(page_title="🛋️ Smart AI Room Decorator", layout="wide")

st.title("🛋️ Smart AI Room Decorator")
st.markdown("""
Upload an image of a room and describe your dream space.
This app **automatically switches models**:
- 🪑 *Preserve layout & furniture*: ControlNet + Canny
- 🏗️ *Redesign empty room*: ControlNet + Depth for better furniture placement
""")

# Set device based on availability
device = "cuda" if torch.cuda.is_available() else "cpu"

# -------------------------------
# 🔧 Load both ControlNet pipelines
# -------------------------------
@st.cache_resource
def load_pipelines():
    # Load Canny-based ControlNet (for preserving layout)
    controlnet_canny = ControlNetModel.from_pretrained(
        "lllyasviel/control_v11p_sd15_canny", torch_dtype=torch.float16
    )
    pipe_canny = StableDiffusionControlNetImg2ImgPipeline.from_pretrained(
        "SG161222/Realistic_Vision_V5.1_noVAE",
        controlnet=controlnet_canny,
        torch_dtype=torch.float16
    ).to(device)

    # Load Depth-based ControlNet (for full room redesigns)
    controlnet_depth = ControlNetModel.from_pretrained(
        "lllyasviel/control_v11f1p_sd15_depth", torch_dtype=torch.float16
    )
    pipe_depth = StableDiffusionControlNetImg2ImgPipeline.from_pretrained(
        "SG161222/Realistic_Vision_V5.1_noVAE",
        controlnet=controlnet_depth,
        torch_dtype=torch.float16
    ).to(device)

    return pipe_canny, pipe_depth

# Load both pipelines
pipe_canny, pipe_depth = load_pipelines()

# ----------------------------------
# 🧠 Helper: Generate Canny edge map
# ----------------------------------
def get_canny_image(image, size=(512, 512)):
    image_resized = image.resize(size)
    image_np = np.array(image_resized)
    edges = cv2.Canny(image_np, 100, 200)
    edges_rgb = np.stack([edges] * 3, axis=-1)  # convert to 3-channel RGB
    return Image.fromarray(edges_rgb).convert("RGB")

# -----------------------------------
# 🧠 Helper: Generate Depth map image
# -----------------------------------
def get_depth_image(image, size=(512, 512)):
    from transformers import DPTFeatureExtractor, DPTForDepthEstimation

    # Load depth model from HuggingFace
    model = DPTForDepthEstimation.from_pretrained("Intel/dpt-hybrid-midas").to(device)
    processor = DPTFeatureExtractor.from_pretrained("Intel/dpt-hybrid-midas")

    image_resized = image.resize(size)
    inputs = processor(images=image_resized, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model(**inputs)
        depth = outputs.predicted_depth.squeeze().cpu().numpy()

    # Normalize and convert to RGB
    depth = cv2.normalize(depth, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    depth_rgb = np.stack([depth]*3, axis=-1)
    return Image.fromarray(depth_rgb).convert("RGB")

# -----------------------------------
# 📤 User Inputs
# -----------------------------------
uploaded_file = st.file_uploader("📤 Upload a room image (JPG or PNG)", type=["jpg", "jpeg", "png"])
design_mode = st.radio("🎯 Choose design goal:", ("🪑 Preserve layout & furniture", "🏗️ Fill empty room with furniture"))
prompt = st.text_input("📝 Describe your dream room:")

# -----------------------------------
# 🔁 Generate Redesign (once image + prompt are ready)
# -----------------------------------
if uploaded_file and prompt:
    original_image = Image.open(uploaded_file).convert("RGB")

    # Choose control method and pipeline
    with st.spinner("🔄 Generating control image..."):
        if design_mode == "🪑 Preserve layout & furniture":
            control_image = get_canny_image(original_image)
            pipe = pipe_canny
            strength = 0.75  # keep structure
        else:
            control_image = get_depth_image(original_image)
            pipe = pipe_depth
            strength = 1.0  # full creativity

    # Generate redesigned room
    with st.spinner("🎨 Generating AI room redesign..."):
        result = pipe(
            prompt=prompt,
            image=original_image.resize((512, 512)).convert("RGB"),
            control_image=control_image.resize((512, 512)).convert("RGB"),
            strength=strength,
            guidance_scale=7.5,
            num_inference_steps=30
        ).images[0]

    # --------------------------
    # 🎭 Display before & after
    # --------------------------
    st.markdown("### 🎭 Before and After")
    col1, col2 = st.columns(2)
    with col1:
        st.markdown("**🖼️ Original Room**")
        st.image(original_image.resize((512, 512)), use_container_width=True)
    with col2:
        st.markdown("**✨ Redesigned Room**")
        st.image(result.resize((512, 512)), use_container_width=True)


Overwriting app.py


In [ ]:
from pyngrok import ngrok
import time
import os

# ✅ Set your ngrok authtoken
NGROK_AUTH_TOKEN = "2zNLmftrSpRCnguytWvDaiIpqba_2ftvNENtR9A5uXLCRvawk"
os.system(f"ngrok config add-authtoken {NGROK_AUTH_TOKEN}")

# 🧹 Kill previous tunnels (avoid duplicate tunnels)
ngrok.kill()

# 🧾 Optional: Clean logs
if os.path.exists("/content/log.txt"):
    os.remove("/content/log.txt")

# 🚀 Run Streamlit in background and log output
print("⏳ Starting Streamlit app...")
os.system("streamlit run app.py &> /content/log.txt &")

# ⏱ Wait for Streamlit to boot up
time.sleep(10)  # <- increase this to 10+ seconds if models are heavy

# 🌍 Start tunnel
print("🔌 Connecting via ngrok...")
public_url = ngrok.connect(8501)

print("✅ App is live! Visit:")
print(public_url)


⏳ Starting Streamlit app...
🔌 Connecting via ngrok...
✅ App is live! Visit:
NgrokTunnel: "https://9e9d-34-143-130-20.ngrok-free.app" -> "http://localhost:8501"
